In [1]:
import sys

sys.path.insert(1, '../input/gpt2source/gpt2-pytorch/pytorch-pretrained-BERT-master')
! pip install ../input/gpt2source/gpt2-pytorch/pytorch-pretrained-BERT-master --upgrade


Processing /kaggle/input/gpt2source/gpt2-pytorch/pytorch-pretrained-BERT-master
  Stored in directory: /tmp/.cache/pip/wheels/fe/7c/95/fccc2929ca759896e080450ab3066b1dd5b6b244f3d818c636
Successfully built pytorch-pretrained-bert
ERROR: allennlp 0.8.4 requires awscli>=1.11.91, which is not installed.
ERROR: allennlp 0.8.4 requires flaky, which is not installed.
ERROR: allennlp 0.8.4 requires responses>=0.7, which is not installed.
  Found existing installation: pytorch-pretrained-bert 0.6.2
    Uninstalling pytorch-pretrained-bert-0.6.2:
      Successfully uninstalled pytorch-pretrained-bert-0.6.2


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import torch.utils.data
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook
import os
import warnings
import gc

### CONFLICT ALERT if used the two lines below
#package_dir = "../input/ppbert/pytorch-pretrained-bert/pytorch-pretrained-BERT"
#sys.path.append(package_dir)
from pytorch_pretrained_bert import GPT2Tokenizer, GPT2ClassificationHeadModel
from pytorch_pretrained_bert import GPT2Config
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam, OpenAIAdam
from pytorch_pretrained_bert import BertConfig
from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
import shutil
warnings.filterwarnings(action='once')
device = torch.device('cuda')

In [3]:
def convert_lines_gpt2(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
        one_token = tokenizer.convert_tokens_to_ids(tokens_a) + [0]*(max_seq_length- len(tokens_a))
        all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)

def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    return np.array(all_tokens)

def convert_lines_both(example, bert_tokenizer, gpt2_tokenizer, bert_max_seq_length=320, gpt2_max_seq_length=220):
    
    bert_max_seq_length -= 2
    gpt2_max_seq_length -= 2
    bert_all_tokens, gpt2_all_tokens = [], []
    #longer = 0
    for text in tqdm(example):
        bert_tokens_a = bert_tokenizer.tokenize(text)
        gpt2_tokens_a = gpt2_tokenizer.tokenize(text)
        if len(bert_tokens_a) > bert_max_seq_length:
            bert_tokens_a = bert_tokens_a[:bert_max_seq_length]
        if len(gpt2_tokens_a) > gpt2_max_seq_length:
            gpt2_tokens_a = gpt2_tokens_a[:gpt2_max_seq_length]
        bert_one_token = bert_tokenizer.convert_tokens_to_ids(bert_tokens_a) + [0]*(bert_max_seq_length- len(bert_tokens_a))
        gpt2_one_token = gpt2_tokenizer.convert_tokens_to_ids(["[CLS]"]+gpt2_tokens_a+["[SEP]"])+[0] * (gpt2_max_seq_length-len(gpt2_tokens_a))
        bert_all_tokens.append(bert_one_token)
        gpt2_all_tokens.append(gpt2_one_token) 
    return np.array(bert_all_tokens), np.array(gpt2_all_tokens)

In [4]:
%%time
#MAX_SEQUENCE_LENGTH = 220
#MAX_SEQUENCE_LENGTH = 256
MAX_SEQUENCE_LENGTH = 320
SEED = 1234
BATCH_SIZE = 32
BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

bert_config = BertConfig(BERT_MODEL_PATH + 'bert_config.json')
bert_tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)

gpt2_config = GPT2Config('../input/gpt2models/gpt2-models/config.json')
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('../input/gpt2models/gpt2-models/')


CPU times: user 104 ms, sys: 24 ms, total: 128 ms
Wall time: 134 ms


../input/gpt2source/gpt2-pytorch/pytorch-pretrained-BERT-master/pytorch_pretrained_bert/tokenization_gpt2.py:146: ResourceWarning: unclosed file <_io.TextIOWrapper name='../input/gpt2models/gpt2-models/vocab.json' mode='r' encoding='UTF-8'>
  self.encoder = json.load(open(vocab_file))
../input/gpt2source/gpt2-pytorch/pytorch-pretrained-BERT-master/pytorch_pretrained_bert/tokenization_gpt2.py:151: ResourceWarning: unclosed file <_io.TextIOWrapper name='../input/gpt2models/gpt2-models/merges.txt' mode='r' encoding='utf-8'>
  bpe_data = open(merges_file, encoding='utf-8').read().split('\n')[1:-1]


In [5]:
%%time
!mkdir ../input/working
!mkdir ../input/working2

BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'
WORK_DIR  = '../input/working/'
convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
    BERT_MODEL_PATH + 'bert_model.ckpt',
BERT_MODEL_PATH + 'bert_config.json',
WORK_DIR + 'pytorch_model.bin')
shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')

BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-24_h-1024_a-16/uncased_L-24_H-1024_A-16/'  
WORK_DIR  = '../input/working2/'
convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
    BERT_MODEL_PATH + 'bert_model.ckpt',
BERT_MODEL_PATH + 'bert_config.json',
WORK_DIR + 'pytorch_model.bin')
shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')

Building PyTorch model from configuration: {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

Converting TensorFlow checkpoint from /kaggle/input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/bert_model.ckpt
Loading TF weight bert/embeddings/LayerNorm/beta with shape [768]
Loading TF weight bert/embeddings/LayerNorm/gamma with shape [768]
Loading TF weight bert/embeddings/position_embeddings with shape [512, 768]
Loading TF weight bert/embeddings/token_type_embeddings with shape [2, 768]
Loading TF weight bert/embeddings/word_embeddings with shape [30522, 768]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/beta with shape [768]
Loading TF weight bert/encoder

Loading TF weight bert/encoder/layer_4/attention/output/LayerNorm/beta with shape [768]
Loading TF weight bert/encoder/layer_4/attention/output/LayerNorm/gamma with shape [768]
Loading TF weight bert/encoder/layer_4/attention/output/dense/bias with shape [768]
Loading TF weight bert/encoder/layer_4/attention/output/dense/kernel with shape [768, 768]
Loading TF weight bert/encoder/layer_4/attention/self/key/bias with shape [768]
Loading TF weight bert/encoder/layer_4/attention/self/key/kernel with shape [768, 768]
Loading TF weight bert/encoder/layer_4/attention/self/query/bias with shape [768]
Loading TF weight bert/encoder/layer_4/attention/self/query/kernel with shape [768, 768]
Loading TF weight bert/encoder/layer_4/attention/self/value/bias with shape [768]
Loading TF weight bert/encoder/layer_4/attention/self/value/kernel with shape [768, 768]
Loading TF weight bert/encoder/layer_4/intermediate/dense/bias with shape [3072]
Loading TF weight bert/encoder/layer_4/intermediate/dense/

Building PyTorch model from configuration: {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

Converting TensorFlow checkpoint from /kaggle/input/bert-pretrained-models/uncased_l-24_h-1024_a-16/uncased_L-24_H-1024_A-16/bert_model.ckpt
Loading TF weight bert/embeddings/LayerNorm/beta with shape [1024]
Loading TF weight bert/embeddings/LayerNorm/gamma with shape [1024]
Loading TF weight bert/embeddings/position_embeddings with shape [512, 1024]
Loading TF weight bert/embeddings/token_type_embeddings with shape [2, 1024]
Loading TF weight bert/embeddings/word_embeddings with shape [30522, 1024]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/beta with shape [1024]
Loading TF weight ber

Loading TF weight bert/encoder/layer_13/output/LayerNorm/gamma with shape [1024]
Loading TF weight bert/encoder/layer_13/output/dense/bias with shape [1024]
Loading TF weight bert/encoder/layer_13/output/dense/kernel with shape [4096, 1024]
Loading TF weight bert/encoder/layer_14/attention/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_14/attention/output/LayerNorm/gamma with shape [1024]
Loading TF weight bert/encoder/layer_14/attention/output/dense/bias with shape [1024]
Loading TF weight bert/encoder/layer_14/attention/output/dense/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_14/attention/self/key/bias with shape [1024]
Loading TF weight bert/encoder/layer_14/attention/self/key/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_14/attention/self/query/bias with shape [1024]
Loading TF weight bert/encoder/layer_14/attention/self/query/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_14/attention/

Loading TF weight bert/encoder/layer_2/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_2/output/LayerNorm/gamma with shape [1024]
Loading TF weight bert/encoder/layer_2/output/dense/bias with shape [1024]
Loading TF weight bert/encoder/layer_2/output/dense/kernel with shape [4096, 1024]
Loading TF weight bert/encoder/layer_20/attention/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_20/attention/output/LayerNorm/gamma with shape [1024]
Loading TF weight bert/encoder/layer_20/attention/output/dense/bias with shape [1024]
Loading TF weight bert/encoder/layer_20/attention/output/dense/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_20/attention/self/key/bias with shape [1024]
Loading TF weight bert/encoder/layer_20/attention/self/key/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_20/attention/self/query/bias with shape [1024]
Loading TF weight bert/encoder/layer_20/attention/self/query/kerne

Loading TF weight bert/encoder/layer_6/attention/self/query/bias with shape [1024]
Loading TF weight bert/encoder/layer_6/attention/self/query/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_6/attention/self/value/bias with shape [1024]
Loading TF weight bert/encoder/layer_6/attention/self/value/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_6/intermediate/dense/bias with shape [4096]
Loading TF weight bert/encoder/layer_6/intermediate/dense/kernel with shape [1024, 4096]
Loading TF weight bert/encoder/layer_6/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_6/output/LayerNorm/gamma with shape [1024]
Loading TF weight bert/encoder/layer_6/output/dense/bias with shape [1024]
Loading TF weight bert/encoder/layer_6/output/dense/kernel with shape [4096, 1024]
Loading TF weight bert/encoder/layer_7/attention/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_7/attention/output/LayerNorm/gamma with

CPU times: user 11 s, sys: 4.92 s, total: 15.9 s
Wall time: 19.9 s


In [6]:
%%time
test_df = pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv")
test_df['comment_text'] = test_df['comment_text'].astype(str) 

X_test_bert, X_test_gpt2 = convert_lines_both(test_df["comment_text"].fillna("DUMMY_VALUE"), bert_tokenizer, gpt2_tokenizer)

100%|██████████| 97320/97320 [02:38<00:00, 614.36it/s]


CPU times: user 2min 40s, sys: 2.39 s, total: 2min 43s
Wall time: 2min 45s


In [8]:
X_test240 = X_test_bert[:,:240]
X_test220 = X_test_bert[:,:220]


In [ ]:
# LB-wise this is better

model = torch.load("../input/fastai-bert-full-3-epoch/fastai_bert_full_3_epoch.model")

model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()
test_preds = np.zeros((len(X_test240)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test240, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=256, shuffle=False)
tk0 = tqdm(test_loader)
for i, (x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds[i * 256:(i + 1) * 256] = pred[:, 0].detach().cpu().squeeze().numpy()

fastai_pred_0 = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'pytorch_pretrained_bert.modeling.BertForSequenceClassification' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.6/site-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'pytorch_pretrained_bert.modeling.BertEmbeddings' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.6/site-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'pytorch_pretrained_bert.modeling.BertSelfOutput' has changed. you can retrieve the original sou

In [ ]:
model = BertForSequenceClassification.from_pretrained('../input/working', num_labels=6)
model.load_state_dict(torch.load("../input/jin-fastai-weights/jin_fastai_fold_0.bin"))

model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()
test_preds = np.zeros((len(X_test240)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test240, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=64, shuffle=False)
tk0 = tqdm(test_loader)
for i, (x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds[i * 64:(i + 1) * 64] = pred[:, 0].detach().cpu().squeeze().numpy()

fastai_pred_1 = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

In [ ]:
model = BertForSequenceClassification.from_pretrained('../input/working', num_labels=7)
model.load_state_dict(torch.load("../input/openai-models/fold_2_320_openai.bin"))

model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

test_preds = np.zeros((len(X_test)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)
tk0 = tqdm(test_loader)
for i, (x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

base_320_0 = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

In [ ]:
model = BertForSequenceClassification.from_pretrained('../input/working', num_labels=7)
model.load_state_dict(torch.load("../input/openai-models/fold_3_320_openai.bin"))

model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

test_preds = np.zeros((len(X_test)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)
tk0 = tqdm(test_loader)
for i, (x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

base_320_1 = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

In [ ]:
## LARGE

#BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-24_h-1024_a-16/uncased_L-24_H-1024_A-16/'
#bert_config = BertConfig(BERT_MODEL_PATH + 'bert_config.json')
#model = BertForSequenceClassification(bert_config, num_labels=7)
model = BertForSequenceClassification.from_pretrained('../input/working2', num_labels=7)

model.load_state_dict(torch.load("../input/bert-large-b/bert_large_fold_0_lr_2e-05_STAGE_4.bin"))

model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

bs = 32
test_preds = np.zeros((len(X_test220)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test220, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=bs, shuffle=False)
tk0 = tqdm(test_loader)
for i, (x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds[i * bs:(i + 1) * bs] = pred[:, 0].detach().cpu().squeeze().numpy()

large_320 = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()


In [ ]:
## GPT2

MAX_SEQUENCE_LENGTH = 220


#X_test_gpt = convert_lines_gpt2(test_df["comment_text"].fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, tokenizer)


In [ ]:

model = GPT2ClassificationHeadModel(gpt2_config, clf_dropout=0.4, n_class=8)
model.load_state_dict(torch.load("../input/jin-gpt2-models/fold_1_220_gpt2.bin"))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()
test_preds = np.zeros((len(X_test_gpt)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test_gpt, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)
tk0 = tqdm(test_loader)
for i, (x_batch,) in enumerate(tk0):
    pred = model.forward(x_batch.to(device))
    test_preds[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

gpt_0 = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()


In [ ]:
#final_pred = 0.15 * fastai_pred + 0.85*( 0.21*(pred_320_0 + pred_320_1 + pred_320_2 + pred_320_4) + 0.16*(pred_220_3) )
#final_pred = 0.15*fastai_pred + 0.85*(0.2*(pred_320_0 + pred_320_1 + pred_320_2 + pred_320_3 + pred_320_4))

final_pred = (0.2*fastai_pred_0 + 0.1*fastai_pred_1 + 0.2*(base_320_0 + base_320_1) + 0.4*large_320 + 0.2*gpt_0) / 1.3


In [ ]:
submission = pd.DataFrame.from_dict({
    'id': test_df['id'],
    #'prediction': test_pred
    'prediction': final_pred
})
submission.to_csv('submission.csv', index=False)